In [1]:
pip install torch torchvision numpy pandas scikit-learn pillow tqdm matplotlib opencv-python


Active code page: 1252
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\satya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
# ========================================
# 1️⃣ Import Libraries
# ========================================
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import cv2


In [7]:
# ========================================
# ✅ Correct Final Dataset Paths
# ========================================
BASE_DIR = r"C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\dataset"

TRAIN_IMG_DIR = os.path.join(BASE_DIR, "Train", "images")
VAL_IMG_DIR = os.path.join(BASE_DIR, "Validation", "images")

TRAIN_CSV = os.path.join(BASE_DIR, "Train", "sliders.csv")
VAL_CSV = os.path.join(BASE_DIR, "Validation", "sliders_input.csv")

print("✅ Train images path:", TRAIN_IMG_DIR)
print("✅ Validation images path:", VAL_IMG_DIR)
print("✅ Train CSV path:", TRAIN_CSV)
print("✅ Validation CSV path:", VAL_CSV)


✅ Train images path: C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\dataset\Train\images
✅ Validation images path: C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\dataset\Validation\images
✅ Train CSV path: C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\dataset\Train\sliders.csv
✅ Validation CSV path: C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\dataset\Validation\sliders_input.csv


In [8]:
# ========================================
# 🔍 Verify Folder Existence
# ========================================
print("Train Images Found:", os.path.exists(TRAIN_IMG_DIR))
print("Validation Images Found:", os.path.exists(VAL_IMG_DIR))
print("Train CSV Found:", os.path.exists(TRAIN_CSV))
print("Validation CSV Found:", os.path.exists(VAL_CSV))

# Also print counts
print("No. of train images:", len(os.listdir(TRAIN_IMG_DIR)))
print("No. of validation images:", len(os.listdir(VAL_IMG_DIR)))


Train Images Found: True
Validation Images Found: True
Train CSV Found: True
Validation CSV Found: True
No. of train images: 2539
No. of validation images: 493


In [10]:
# ========================================
# 🕵️ Step 1 — Check available columns in your CSV
# ========================================
import pandas as pd

check_train = pd.read_csv(TRAIN_CSV)
print("Available Columns in Train CSV:\n", list(check_train.columns))


Available Columns in Train CSV:
 ['copyCreationTime', 'captureTime', 'touchTime', 'id_global', 'grayscale', 'aperture', 'flashFired', 'focalLength', 'isoSpeedRating', 'shutterSpeed', 'Temperature', 'Tint', 'currTemp', 'currTint']


In [11]:
# ========================================
# Load and Preprocess Metadata
# ========================================

import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load CSVs
train_df = pd.read_csv(TRAIN_CSV)
val_df = pd.read_csv(VAL_CSV)

# Define columns
numeric_cols = [
    'grayscale', 'aperture', 'flashFired', 
    'focalLength', 'isoSpeedRating', 'shutterSpeed', 
    'currTemp', 'currTint'
]
categorical_cols = []  # none in your dataset

# Fill missing numeric values
train_df[numeric_cols] = train_df[numeric_cols].fillna(train_df[numeric_cols].mean())
val_df[numeric_cols] = val_df[numeric_cols].fillna(train_df[numeric_cols].mean())

# Normalize numeric columns
scaler = StandardScaler()
train_df[numeric_cols] = scaler.fit_transform(train_df[numeric_cols])
val_df[numeric_cols] = scaler.transform(val_df[numeric_cols])

print("✅ Preprocessing successful!")
print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)
print("Numeric columns used:", numeric_cols)


✅ Preprocessing successful!
Train shape: (2538, 14)
Val shape: (493, 12)
Numeric columns used: ['grayscale', 'aperture', 'flashFired', 'focalLength', 'isoSpeedRating', 'shutterSpeed', 'currTemp', 'currTint']


In [ ]:
# ========================================
# 📦 Step 2 — Custom Dataset (Final)
# ========================================
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch
import os

class AftershootDataset(Dataset):
    def __init__(self, df, img_dir, numeric_cols, is_train=True):
        self.df = df
        self.img_dir = img_dir
        self.numeric_cols = numeric_cols
        self.is_train = is_train
        
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{row['id_global']}.tif")
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        
        metadata = torch.tensor(row[self.numeric_cols].values, dtype=torch.float32)
        
        if self.is_train:
            label = torch.tensor([row['Temperature'], row['Tint']], dtype=torch.float32)
            return image, metadata, label
        else:
            return image, metadata, row['id_global']


In [15]:
# ========================================
# 🧩 Step 3 — Create Train + Validation Datasets
# ========================================
from torch.utils.data import DataLoader

train_dataset = AftershootDataset(train_df, TRAIN_IMG_DIR, numeric_cols, is_train=True)
val_dataset = AftershootDataset(val_df, VAL_IMG_DIR, numeric_cols, is_train=False)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
print("✅ Dataloader ready — batches:", len(train_loader))


✅ Dataloader ready — batches: 159


In [16]:
# ========================================
# 🧠 Step 4 — Hybrid Model Definition
# ========================================
import torch.nn as nn
from torchvision import models

class AftershootModel(nn.Module):
    def __init__(self, metadata_dim):
        super(AftershootModel, self).__init__()
        
        # CNN backbone (ResNet18)
        self.cnn = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.cnn.fc = nn.Identity()  # Remove classifier layer
        
        # MLP for metadata
        self.mlp = nn.Sequential(
            nn.Linear(metadata_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        
        # Fusion + output layer
        self.fc = nn.Sequential(
            nn.Linear(512 + 64, 128),
            nn.ReLU(),
            nn.Linear(128, 2)  # Predict Temperature and Tint
        )
    
    def forward(self, img, meta):
        img_feat = self.cnn(img)
        meta_feat = self.mlp(meta)
        combined = torch.cat([img_feat, meta_feat], dim=1)
        output = self.fc(combined)
        return output


In [18]:
# ========================================
# 🧹 Fix Mixed Data Types in Numeric Columns
# ========================================

def clean_numeric(df, cols):
    for col in cols:
        # Convert all values to numeric (invalid ones become NaN)
        df[col] = pd.to_numeric(df[col], errors='coerce')
        # Fill NaN with column mean
        df[col].fillna(df[col].mean(), inplace=True)
    return df

train_df = clean_numeric(train_df, numeric_cols)
val_df = clean_numeric(val_df, numeric_cols)

print("✅ All numeric columns cleaned and converted to float.")
print(train_df[numeric_cols].dtypes)


✅ All numeric columns cleaned and converted to float.
grayscale         float64
aperture          float64
flashFired        float64
focalLength       float64
isoSpeedRating    float64
shutterSpeed      float64
currTemp          float64
currTint          float64
dtype: object


C:\Users\satya\AppData\Local\Temp\ipykernel_9276\2258300746.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [20]:
# ========================================
# 📦 Step 2 — Safe Dataset (handles all float conversions)
# ========================================
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch
import numpy as np
import os

class AftershootDataset(Dataset):
    def __init__(self, df, img_dir, numeric_cols, is_train=True):
        self.df = df
        self.img_dir = img_dir
        self.numeric_cols = numeric_cols
        self.is_train = is_train
        
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{row['id_global']}.tif")
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        
        # Convert metadata safely to float32 tensor
        metadata = np.array(row[self.numeric_cols].astype(float).values, dtype=np.float32)
        metadata = torch.tensor(metadata, dtype=torch.float32)
        
        if self.is_train:
            label = torch.tensor([float(row['Temperature']), float(row['Tint'])], dtype=torch.float32)
            return image, metadata, label
        else:
            return image, metadata, row['id_global']


In [21]:
train_dataset = AftershootDataset(train_df, TRAIN_IMG_DIR, numeric_cols, is_train=True)
val_dataset = AftershootDataset(val_df, VAL_IMG_DIR, numeric_cols, is_train=False)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
print("✅ Safe DataLoader ready — batches:", len(train_loader))


✅ Safe DataLoader ready — batches: 159


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

model = AftershootModel(metadata_dim=len(numeric_cols)).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    
    for img, meta, label in tqdm(train_loader):
        img, meta, label = img.to(device), meta.to(device), label.to(device)
        
        optimizer.zero_grad()
        outputs = model(img, meta)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{EPOCHS}] — Loss: {running_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), "aftershoot_model.pth")
print("✅ Model trained & saved successfully!")


✅ Using device: cpu


100%|██████████| 159/159 [06:18<00:00,  2.38s/it]


Epoch [1/5] — Loss: 2510.0388


100%|██████████| 159/159 [05:34<00:00,  2.10s/it]


Epoch [2/5] — Loss: 2468.8793


...

Epoch [3/5] — Loss: 2355.7002


100%|██████████| 159/159 [05:39<00:00,  2.13s/it]


Epoch [4/5] — Loss: 2132.9934


100%|██████████| 159/159 [05:25<00:00,  2.05s/it]


Epoch [5/5] — Loss: 1749.0506
✅ Model trained & saved successfully!


In [23]:
# ========================================
# 🧾 Step 6 — Inference (Generate Predictions)
# ========================================
from torch.utils.data import DataLoader
import pandas as pd
from tqdm import tqdm

# Load trained model weights
model = AftershootModel(metadata_dim=len(numeric_cols)).to(device)
model.load_state_dict(torch.load("aftershoot_model.pth", map_location=device))
model.eval()

# Create validation DataLoader
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)

predictions = []

with torch.no_grad():
    for img, meta, ids in tqdm(val_loader):
        img, meta = img.to(device), meta.to(device)
        outputs = model(img, meta)
        outputs = outputs.cpu().numpy()
        
        for i, id_ in enumerate(ids):
            temp, tint = outputs[i]
            predictions.append([id_, round(float(temp)), round(float(tint))])

# Convert to DataFrame and save
pred_df = pd.DataFrame(predictions, columns=['id_global', 'Temperature', 'Tint'])
output_path = r"C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\submissions\predictions.csv"
pred_df.to_csv(output_path, index=False)

print(f"✅ Predictions saved successfully at:\n{output_path}")
print(f"Total predictions generated: {len(pred_df)}")


100%|██████████| 31/31 [00:27<00:00,  1.12it/s]

✅ Predictions saved successfully at:
C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\submissions\predictions.csv
Total predictions generated: 493


In [24]:
check_pred = pd.read_csv(output_path)
print(check_pred.head())
print("Shape:", check_pred.shape)


                              id_global  Temperature  Tint
0  EB5BEE31-8D4F-450A-8BDD-27C762C75AA6         2748    11
1  DE666E1F-0433-4958-AEC0-9A0CC0F81036         2590    10
2  F6A6EA9C-A5C2-4BBA-9812-5CE52B818CB6         2655    11
3  BCC39DEF-598C-491A-A3CA-14A249717F36         2532    10
4  390ED94E-0066-4822-99B9-8F1568BDFBF5         2348     9
Shape: (493, 3)


In [27]:
from sklearn.preprocessing import StandardScaler

# Create target scaler
target_scaler = StandardScaler()

# Fit on train targets and transform
train_df[['Temperature', 'Tint']] = target_scaler.fit_transform(train_df[['Temperature', 'Tint']])

print("✅ Target columns normalized successfully!")


✅ Target columns normalized successfully!


In [28]:
# ========================================
# 🧾 Step 6 — Inference (with inverse scaling)
# ========================================
from torch.utils.data import DataLoader
import pandas as pd
from tqdm import tqdm

# Load trained model
model = AftershootModel(metadata_dim=len(numeric_cols)).to(device)
model.load_state_dict(torch.load("aftershoot_model.pth", map_location=device))
model.eval()

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)

predictions = []

with torch.no_grad():
    for img, meta, ids in tqdm(val_loader):
        img, meta = img.to(device), meta.to(device)
        outputs = model(img, meta)
        outputs = outputs.cpu().numpy()
        
        # inverse-transform normalized outputs
        outputs = target_scaler.inverse_transform(outputs)

        for i, id_ in enumerate(ids):
            temp, tint = outputs[i]
            predictions.append([id_, round(float(temp)), round(float(tint))])

# Save predictions
output_path = r"C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\submissions\predictions.csv"
pd.DataFrame(predictions, columns=['id_global', 'Temperature', 'Tint']).to_csv(output_path, index=False)

print(f"✅ Predictions saved successfully at:\n{output_path}")


100%|██████████| 31/31 [00:24<00:00,  1.26it/s]

✅ Predictions saved successfully at:
C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\submissions\predictions.csv


In [29]:
# ========================================
# 🚀 Step 5 — Improved Training Configuration
# ========================================
import torch
from torch import nn, optim
from tqdm import tqdm

model = AftershootModel(metadata_dim=len(numeric_cols)).to(device)

criterion = nn.L1Loss()  # MAE
optimizer = optim.Adam(model.parameters(), lr=5e-5)  # smaller LR

EPOCHS = 20  # increased training duration

print(f"✅ Using device: {device}")
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    
    for img, meta, label in tqdm(train_loader):
        img, meta, label = img.to(device), meta.to(device), label.to(device)
        
        optimizer.zero_grad()
        outputs = model(img, meta)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{EPOCHS}] — Loss: {avg_loss:.4f}")

torch.save(model.state_dict(), "aftershoot_model.pth")
print("✅ Model retrained & saved successfully (improved version)!")


✅ Using device: cpu


100%|██████████| 159/159 [05:53<00:00,  2.23s/it]


Epoch [1/20] — Loss: 0.5475


100%|██████████| 159/159 [06:16<00:00,  2.37s/it]


Epoch [2/20] — Loss: 0.4188


...

Epoch [18/20] — Loss: 0.1607


100%|██████████| 159/159 [05:32<00:00,  2.09s/it]


Epoch [19/20] — Loss: 0.1544


100%|██████████| 159/159 [05:37<00:00,  2.12s/it]

Epoch [20/20] — Loss: 0.1502
✅ Model retrained & saved successfully (improved version)!


In [34]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# ✅ Load original training CSV (not preprocessed)
train_df = pd.read_csv(r"C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\dataset\Train\sliders.csv")

# Fit scaler on actual Temperature/Tint columns (real values, not normalized)
target_scaler = StandardScaler()
target_scaler.fit(train_df[['Temperature', 'Tint']])

# Reload trained model
model = AftershootModel(metadata_dim=len(numeric_cols)).to(device)
model.load_state_dict(torch.load("aftershoot_model.pth", map_location=device))
model.eval()

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)

predictions = []
print("🚀 Running inference with correct scaling...")

with torch.no_grad():
    for img, meta, ids in tqdm(val_loader):
        img, meta = img.to(device), meta.to(device)
        outputs = model(img, meta)
        outputs = outputs.cpu().numpy()

        # Inverse scale properly using original target range
        outputs = target_scaler.inverse_transform(outputs)

        for i, id_ in enumerate(ids):
            temp, tint = outputs[i]
            predictions.append([id_, round(float(temp)), round(float(tint))])

# Save final predictions
output_path = r"C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\submissions\predictions.csv"
pd.DataFrame(predictions, columns=['id_global', 'Temperature', 'Tint']).to_csv(output_path, index=False)

print(f"✅ Final predictions saved at:\n{output_path}")


🚀 Running inference with correct scaling...


100%|██████████| 31/31 [00:25<00:00,  1.21it/s]

✅ Final predictions saved at:
C:\Users\satya\Downloads\Aftershoot_AI_Challenge\task\submissions\predictions.csv
